In [10]:
import optuna as op
%matplotlib inline
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_FunkSVD_Cython
from Utils.createURM import createURM

In [11]:
URM = createURM()

C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Utils\createURM.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('../Input/interactions_and_impressions.csv')


In [12]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

In [13]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 716 ( 1.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 324 ( 0.8%) Users that have less than 1 test interactions


## FunkSVD

In [14]:
recommender = MatrixFactorization_FunkSVD_Cython(URM_train)

def objective(trial):
    num_factors = trial.suggest_float("num_factors", 50, 500)
    user_reg = trial.suggest_float("user_reg", 1, 10)
    item_reg = trial.suggest_float("item_reg", 1, 10)
    recommender.fit(
            epochs=10,
            batch_size=1000,
            num_factors=num_factors,
            positive_threshold_BPR=None,
            learning_rate=0.001,
            use_bias=False,
            sgd_mode='adam',
            negative_interactions_quota=0.0,
            init_mean=0.0, init_std_dev=0.1,
            user_reg=user_reg, item_reg=item_reg, bias_reg=0.0, positive_reg=0.0, negative_reg=0.0,
            random_seed=None,
            **{
                'epochs_min' : 0,
                'evaluator_object' : evaluator_validation,
                'stop_on_validation' : True,
                'validation_every_n' : 5,
                'validation_metric' : 'MAP',
                'lower_validations_allowed' : 3
            }
    )

    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    return result_dict.loc[10]["MAP"]

In [15]:
study = op.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

[I 2022-11-06 14:41:17,789] A new study created in memory with name: no-name-0c935b0d-9243-4316-af2c-5ebbefe8f930


FUNK_SVD: Processed 995000 (100.0%) in 21.98 sec. MSE loss 1.70E+01. Sample per second: 45259
FUNK_SVD: Epoch 1 of 10. Elapsed time 21.92 sec
FUNK_SVD: Processed 995000 (100.0%) in 22.72 sec. MSE loss 1.67E+01. Sample per second: 43785
FUNK_SVD: Epoch 2 of 10. Elapsed time 43.65 sec
FUNK_SVD: Processed 995000 (100.0%) in 22.48 sec. MSE loss 1.58E+01. Sample per second: 44253
FUNK_SVD: Epoch 3 of 10. Elapsed time 1.09 min
FUNK_SVD: Processed 995000 (100.0%) in 23.03 sec. MSE loss 1.34E+01. Sample per second: 43201
FUNK_SVD: Epoch 4 of 10. Elapsed time 1.47 min
FUNK_SVD: Processed 995000 (100.0%) in 22.00 sec. MSE loss 1.03E+01. Sample per second: 45226
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 40913 (100.0%) in 29.55 sec. Users per second: 1384
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0178770, PRECISION_RECALL_MIN_DEN: 0.0293527, RECALL: 0.0274167, MAP: 0.0068349, MAP_MIN_DEN: 0.0111769, MRR: 0.0559141, NDCG: 0.0276888, F1: 0.0216422, HIT_RATE: 0.1472148, ARHR_ALL_HITS: 0.06

[I 2022-11-06 14:46:31,622] Trial 0 finished with value: 0.0068348865711563185 and parameters: {'num_factors': 141.42397448002794, 'user_reg': 6.569205321080409, 'item_reg': 4.568108884230745}. Best is trial 0 with value: 0.0068348865711563185.


FUNK_SVD: Processed 995000 (100.0%) in 30.02 sec. MSE loss 1.69E+01. Sample per second: 33145
FUNK_SVD: Epoch 1 of 10. Elapsed time 29.97 sec
FUNK_SVD: Processed 995000 (100.0%) in 29.96 sec. MSE loss 1.65E+01. Sample per second: 33206
FUNK_SVD: Epoch 2 of 10. Elapsed time 59.93 sec
FUNK_SVD: Processed 995000 (100.0%) in 30.03 sec. MSE loss 1.56E+01. Sample per second: 33138
FUNK_SVD: Epoch 3 of 10. Elapsed time 1.48 min
FUNK_SVD: Processed 995000 (100.0%) in 29.09 sec. MSE loss 1.30E+01. Sample per second: 34206
FUNK_SVD: Epoch 4 of 10. Elapsed time 1.97 min
FUNK_SVD: Processed 995000 (100.0%) in 28.63 sec. MSE loss 9.94E+00. Sample per second: 34755
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 40913 (100.0%) in 31.26 sec. Users per second: 1309
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0178941, PRECISION_RECALL_MIN_DEN: 0.0298287, RECALL: 0.0279000, MAP: 0.0068750, MAP_MIN_DEN: 0.0113537, MRR: 0.0565004, NDCG: 0.0283417, F1: 0.0218039, HIT_RATE: 0.1480214, ARHR_ALL_HITS: 0.06

[I 2022-11-06 14:53:00,414] Trial 1 finished with value: 0.006874950291346173 and parameters: {'num_factors': 186.49842762740138, 'user_reg': 6.720283373203484, 'item_reg': 5.639741227630359}. Best is trial 1 with value: 0.006874950291346173.


...

[I 2022-11-06 15:08:34,647] Trial 2 finished with value: 0.006920204079969993 and parameters: {'num_factors': 463.09671291017145, 'user_reg': 9.634779566999654, 'item_reg': 6.562212829181321}. Best is trial 2 with value: 0.006920204079969993.


FUNK_SVD: Processed 995000 (100.0%) in 1.12 min. MSE loss 1.68E+01. Sample per second: 14860
FUNK_SVD: Epoch 1 of 10. Elapsed time 1.11 min
FUNK_SVD: Processed 995000 (100.0%) in 1.11 min. MSE loss 1.61E+01. Sample per second: 14911
FUNK_SVD: Epoch 2 of 10. Elapsed time 2.20 min
FUNK_SVD: Processed 995000 (100.0%) in 1.07 min. MSE loss 1.52E+01. Sample per second: 15470
FUNK_SVD: Epoch 3 of 10. Elapsed time 3.26 min
FUNK_SVD: Processed 995000 (100.0%) in 1.11 min. MSE loss 1.26E+01. Sample per second: 14937
FUNK_SVD: Epoch 4 of 10. Elapsed time 4.37 min
FUNK_SVD: Processed 995000 (100.0%) in 1.11 min. MSE loss 9.88E+00. Sample per second: 14951
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 40913 (100.0%) in 43.28 sec. Users per second: 945
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0184196, PRECISION_RECALL_MIN_DEN: 0.0305470, RECALL: 0.0286139, MAP: 0.0070229, MAP_MIN_DEN: 0.0114354, MRR: 0.0567572, NDCG: 0.0285400, F1: 0.0224119, HIT_RATE: 0.1510767, ARHR_ALL_HITS: 0.0630139, N

[I 2022-11-06 15:21:42,247] Trial 3 finished with value: 0.007022908463526468 and parameters: {'num_factors': 371.21502354206234, 'user_reg': 9.193456056995329, 'item_reg': 3.484514906797387}. Best is trial 3 with value: 0.007022908463526468.


FUNK_SVD: Processed 995000 (100.0%) in 51.52 sec. MSE loss 1.68E+01. Sample per second: 19311
FUNK_SVD: Epoch 1 of 10. Elapsed time 50.53 sec
FUNK_SVD: Processed 995000 (100.0%) in 50.44 sec. MSE loss 1.62E+01. Sample per second: 19725
FUNK_SVD: Epoch 2 of 10. Elapsed time 1.67 min
FUNK_SVD: Processed 995000 (100.0%) in 49.99 sec. MSE loss 1.52E+01. Sample per second: 19903
FUNK_SVD: Epoch 3 of 10. Elapsed time 2.50 min
FUNK_SVD: Processed 995000 (100.0%) in 50.98 sec. MSE loss 1.25E+01. Sample per second: 19517
FUNK_SVD: Epoch 4 of 10. Elapsed time 3.33 min
FUNK_SVD: Processed 995000 (100.0%) in 51.25 sec. MSE loss 9.60E+00. Sample per second: 19413
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 40913 (100.0%) in 42.62 sec. Users per second: 960
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0172708, PRECISION_RECALL_MIN_DEN: 0.0286548, RECALL: 0.0268578, MAP: 0.0065299, MAP_MIN_DEN: 0.0107093, MRR: 0.0535183, NDCG: 0.0270482, F1: 0.0210229, HIT_RATE: 0.1430108, ARHR_ALL_HITS: 0.0590

[I 2022-11-06 15:32:16,403] Trial 4 finished with value: 0.006529904338241613 and parameters: {'num_factors': 285.5816022718442, 'user_reg': 7.669741622305774, 'item_reg': 1.4585326954090112}. Best is trial 3 with value: 0.007022908463526468.


In [16]:
num_factors = study.best_params['num_factors']
user_reg = study.best_params['user_reg']
item_reg = study.best_params['item_reg']

In [17]:
recommender.fit(
            epochs=10,
            batch_size=1000,
            num_factors=num_factors,
            positive_threshold_BPR=None,
            learning_rate=0.001,
            use_bias=False,
            sgd_mode='adam',
            negative_interactions_quota=0.0,
            init_mean=0.0, init_std_dev=0.1,
            user_reg=user_reg, item_reg=item_reg, bias_reg=0.0, positive_reg=0.0, negative_reg=0.0,
            random_seed=None,
            **{
                'epochs_min' : 0,
                'evaluator_object' : evaluator_validation,
                'stop_on_validation' : True,
                'validation_every_n' : 5,
                'validation_metric' : 'MAP',
                'lower_validations_allowed' : 3
            })
result_df, _ = evaluator_test.evaluateRecommender(recommender)
result_df

FUNK_SVD: Processed 995000 (100.0%) in 1.11 min. MSE loss 1.68E+01. Sample per second: 14961
FUNK_SVD: Epoch 1 of 10. Elapsed time 1.10 min
FUNK_SVD: Processed 995000 (100.0%) in 1.11 min. MSE loss 1.61E+01. Sample per second: 14968
FUNK_SVD: Epoch 2 of 10. Elapsed time 2.20 min
FUNK_SVD: Processed 995000 (100.0%) in 1.12 min. MSE loss 1.53E+01. Sample per second: 14752
FUNK_SVD: Epoch 3 of 10. Elapsed time 3.32 min
FUNK_SVD: Processed 995000 (100.0%) in 1.09 min. MSE loss 1.28E+01. Sample per second: 15260
FUNK_SVD: Epoch 4 of 10. Elapsed time 4.40 min
FUNK_SVD: Processed 995000 (100.0%) in 1.20 min. MSE loss 1.00E+01. Sample per second: 13856
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 40913 (100.0%) in 42.40 sec. Users per second: 965
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0185125, PRECISION_RECALL_MIN_DEN: 0.0304934, RECALL: 0.0285360, MAP: 0.0070950, MAP_MIN_DEN: 0.0114115, MRR: 0.0570583, NDCG: 0.0286641, F1: 0.0224565, HIT_RATE: 0.1509300, ARHR_ALL_HITS: 0.0635005, N

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.023464,0.033697,0.030092,0.00927,0.013031,0.072161,0.032863,0.026368,0.18499,0.081576,...,0.992217,0.18355,0.992217,0.032555,6.214462,0.959299,0.064561,0.460632,7.156731,0.294069
